In [3]:
import cv2
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from numpy.linalg import inv
from collections import deque
%matplotlib inline

def mask_region(img):
    """
    Applies an image mask within supplied vertices.
    
    Only masks the region of the image defined by the polygon
    formed from `vertices`. The rest of the image remains untouched
    """
    #defining a blank mask to start with
    mask = np.copy(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (0,) * channel_count
    else:
        ignore_mask_color = 0
    vertices = np.float32([[630,470],[680,470],[1050,720],[250,720]])
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, np.array([vertices], dtype=np.int32), ignore_mask_color)
    return mask


def clear_test_dir():
    test = 'pipelines/frame*'
    r = glob.glob(test)
    for i in r:
        os.remove(i)
        
        
def is_empty_prev_fits():
    global previous_left_fits
    global previous_right_fits
    return len(previous_left_fits) == 0 or len(previous_right_fits) == 0

def append_to_prev_fits(left_fit,right_fit):
    global previous_left_fits
    global previous_right_fits
    previous_left_fits.append(left_fit)
    previous_right_fits.append(right_fit)
    
def clear_prev_fits():
    global previous_left_fits
    global previous_right_fits
    global previous_left_curvs
    global previous_right_curvs
    
    previous_left_fits.clear()
    previous_right_fits.clear()
    previous_left_curvs.clear()
    previous_right_curvs.clear()
    
def average_prev_fits():
    global previous_left_fits
    global previous_right_fits
    left_average = average_cof(previous_left_fits)
    right_average = average_cof(previous_right_fits)
    return left_average,right_average

#average of coefficients
def average_cof(deque):
    average = [0,0,0]
    for elem in deque:
        average [0] += elem[0]
        average [1] += elem[1]
        average [2] += elem[2]    
    size = len(deque)
    if size > 0:
        average[:] = [x / size for x in average]
    return average

def average(deque):
    average = 0    
    for elem in deque:
        average  += elem
    size = len(deque)
    
    if size > 0:
        average /= size
        
    return average


def draw_preview_on_image(background_img,preview_imgs,scale):
    y_offset = 0
    for i in range(0,len(preview_imgs)):        
        if(preview_imgs[i] is not None):
            preview_img = preview_imgs[i]
            height = int(preview_img.shape[0]*scale)
            width = int(preview_img.shape[1]*scale)
            preview_img = cv2.resize(preview_img,(width, height), interpolation = cv2.INTER_CUBIC)
            preview_img = np.dstack((preview_img, preview_img, preview_img))*255   
            x_offset = background_img.shape[1] - ((i +1) *(width + 25))
            background_img[y_offset:y_offset+preview_img.shape[0], x_offset:x_offset+preview_img.shape[1]] = preview_img
            

# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(75, 255), sx_thresh=(90, 140)):
    global src
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    combined_binary = mask_region(combined_binary)  
    return combined_binary


def calibrate_camera():
   
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)    

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    
    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    
    gray_shape = (0,0)

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray_shape = gray.shape[::-1]
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_shape, None, None)
    return mtx,dist
    


       
def undistort(img,mtx,dist):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist


def warper(img, src,offset=125):
    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    dst = np.float32([[offset-25, offset], [img_size[0]-offset+25, offset], 
                                     [img_size[0]-offset, img_size[1]-offset], 
                                     [offset, img_size[1]-offset]])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    return warped, M


def visualize(imgs, labels, num_cols = 4):
    num_rows = (int)(len(imgs)/num_cols) + 1
    plt.close('all')
    fig, ax = plt.subplots(num_rows,num_cols, figsize=(num_cols * 10, num_rows * 10),subplot_kw={'xticks': [], 'yticks': []})
    fig.subplots_adjust(hspace=0.3, wspace=0.05)
    idx = 0
    for img, label in zip(imgs, labels):
        row = (int)(idx/num_cols)
        col = (int)(idx%num_cols)
        if num_cols == 1 :
            slot = ax[row]
        else:
            slot = ax[row,col]
        slot.imshow(img, aspect='auto',cmap='gray')
        slot.text(-1, -1, label, style='italic', bbox={'facecolor':'red', 'alpha':0.5, 'pad':1})
        idx += 1

def window_search(binary_warped):    
       #Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 150
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = [] 
        
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        
    # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx,lefty,rightx,righty

def draw_on_origianl_img(undist, warped, ploty, left_fitx, right_fitx, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0]))    
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

        
def find_lane_lines(binary_warped):
    global left_fit
    global right_fit
    global faulty_frame_count
    global apply_window_search
    global MAX_FAULTY_FRAMES
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    
    leftx,lefty,rightx,righty = find_line_points(binary_warped)
    ret, current_left_fit, current_right_fit = make_valid_lines(leftx, lefty,rightx,righty)
        
    if (ret):
    # Generate x and y values for plotting
        left_fit = current_left_fit
        right_fit = current_right_fit
        append_to_prev_fits(left_fit,right_fit)
        left_fit,right_fit = average_prev_fits()
    else: 
        faulty_frame_count += 1        
        #if fault count = max allowed, do window search again
        if faulty_frame_count == MAX_FAULTY_FRAMES:
            apply_window_search = True
            faulty_frame_count = 0
            return find_lane_lines(binary_warped)
        
        else:
            left_fit,right_fit = average_prev_fits()
         
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return True, ploty, left_fitx, right_fitx

def calculate_curvature(cof,prev_curvs):    
    ym_per_pix = 30.0/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    x_fit = cof[0] * ploty**2 + cof[1] * ploty + cof[2]
    y_eval = np.max(ploty)
    #Fit new polynomials to x,y in world space
    fit_cr = np.polyfit(ploty*ym_per_pix, x_fit*xm_per_pix, 2)
    # Calculate the new radii of curvature
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    #insanly high
    if curverad > 25000:
        return average(prev_curvs)
    return curverad


def calculate_center_offset(shape):
    global left_fit
    global right_fit
    xm_per_pix = 3.7/700
    
    camera_position = shape[1]/2
    point_y = shape[1] -1
    left_point_x = left_fit[0] * point_y**2 + left_fit[1] * point_y + left_fit[2]
    right_point_x = right_fit[0] * point_y**2 + right_fit[1] * point_y + right_fit[2]
    lane_center = (left_point_x + right_point_x)/2
    center_offset_pixels = camera_position - lane_center
    center_offset_meters = xm_per_pix* center_offset_pixels
    return center_offset_meters
    
def find_line_points(binary_warped):    
    global apply_window_search
    global left_fit
    global right_fit  
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 200
    
    if (apply_window_search):
        leftx,lefty,rightx,righty = window_search(binary_warped)
        apply_window_search = False
    else:
        if (len(left_fit) == 0 or len(right_fit) == 0):
            return np.empty([0,0]), np.empty([0,0]), np.empty([0,0]), np.empty([0,0])
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        if(leftx.size == 0 or lefty.size == 0 or rightx.size == 0 or righty.size == 0):
            leftx,lefty,rightx,righty = window_search(binary_warped)        
    
    return leftx,lefty,rightx,righty

def process_image(img):
    global mtx
    global dist
    global src
    
    undistorted = undistort(img,mtx,dist) 
    plined = pipeline(undistorted)
    warped, M = warper(plined,src)
    ret, ploty, left_fitx, right_fitx = find_lane_lines(warped)
    
    if (ret):
        Minv = inv(M)
        result = draw_on_origianl_img(undistorted, warped, ploty, left_fitx, right_fitx, Minv)
    else:
        result = undistorted
        
    #make curvature image
    draw_curvature(result)
    #draw preview of pipeline and warped images
    draw_preview_on_image(result,[plined,warped],0.2)

    return result

def draw_curvature(img):
    global left_fit
    global right_fit
    global previous_left_curvs
    global previous_right_curvs
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_size = 1
    offset = calculate_center_offset(img.shape)
    
    offset_dir = str("{0:.2f}".format(offset))  + ' m ' + 'right of center'
    if offset < 0:
        offset_dir = str("{0:.2f}".format(abs(offset)))  + ' m ' + 'left of center'
    elif offset == 0:
        offset_dir = 'centered'
        
    offset_text = 'Car is ' + offset_dir
    l_text = 'Left curvature: ' + str(int(calculate_curvature(left_fit,previous_left_curvs))) + 'm'
    r_text = 'Right curvature: '+ str(int(calculate_curvature(right_fit,previous_right_curvs))) + 'm'
    
    cv2.putText(img,offset_text,(25,30), font, font_size,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(img,l_text,(25,80), font, font_size,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(img, r_text ,(25,130), font, font_size,(255,255,255),1,cv2.LINE_AA)
    
    
def slope(x1,x2,y1,y2):
    if x2 - x1 == 0:
        return sys.float_info.max    
    slope = ((y2 - y1)/(1.0 * (x2 - x1)) )
    return slope

def parallel_lines(left_fit, right_fit, leftx, lefty,rightx,righty):
    MAX_SLOPE_DIFF = 1000
    m1 = slope(leftx[0], leftx[-1], lefty[0], lefty[-1])
    m2 = slope(rightx[0], rightx[-1], righty[0], righty[-1])
    return abs(m2- m1) < MAX_SLOPE_DIFF

def enough_horz_dist(left_fit, right_fit, leftx, lefty,rightx,righty):
    HORZ_DIST_MIN_TOP = 100
    HORZ_DIST_MIN_BOTTOM = 400
    HORZ_DIST_MAX_TOP = 1000
    HORZ_DIST_MAX_BOTTOM = 2000  
    
    sorted_leftx = sorted(leftx)
    sorted_rightx = sorted(rightx)    
    diff_top = sorted_rightx[0] - sorted_leftx[-1] 
    diff_bottom = sorted_rightx[-1] - sorted_leftx[0] 
    return  diff_bottom > HORZ_DIST_MIN_BOTTOM and diff_top > HORZ_DIST_MIN_TOP and  diff_top < HORZ_DIST_MAX_TOP and diff_bottom < HORZ_DIST_MAX_BOTTOM


def same_curvature(left_fit,right_fit): 
    global previous_left_curvs
    global previous_right_curvs
    # Now our radius of curvature is in meters
    left_cur = calculate_curvature(left_fit,previous_left_curvs)
    right_cur = calculate_curvature(right_fit,previous_right_curvs)
    valid_ratio = False
    if left_cur < right_cur:
        valid_ratio = left_cur * 3 >= right_cur
    else:
        valid_ratio = right_cur * 3 >= left_cur
        
    #168 min radius for 70 KM/H 
    good_curv = valid_ratio and left_cur > 236 and right_cur > 236
    if good_curv:
        previous_left_curvs.append(left_cur)
        previous_right_curvs.append(right_cur)
        return True
    
    return False
        
    


def make_valid_lines(leftx, lefty,rightx,righty):    
    if(leftx.size == 0 or lefty.size == 0 or rightx.size == 0 or righty.size == 0):
        return False, None, None
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    if (parallel_lines(left_fit, right_fit, leftx, lefty,rightx,righty) 
        and enough_horz_dist(left_fit, right_fit, leftx, lefty,rightx,righty)
        and same_curvature(left_fit,right_fit)):
        return True, left_fit, right_fit
    else:
        return False, None, None

    

def test_imgs():    
    global MAX_FAULTY_FRAMES
    MAX_FAULTY_FRAMES = 1
    # Make a list of calibration images
    images = glob.glob('test_images/*')
    imgs = []
    labels = []

    for fname in images:
        img = cv2.imread(fname)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = process_image(img,mtx,dist)
        imgs.append(result)
        labels.append(fname)    

   
    visualize(imgs, labels, 1)

def process_video():
    global MAX_FAULTY_FRAMES
    MAX_FAULTY_FRAMES = 3
    white_output = 'project_output.mp4'
    clip1 = VideoFileClip("project_video.mp4")
    white_clip = clip1.fl_image(process_image)
    %time white_clip.write_videofile(white_output, audio=False)

# define 4 source points src 
src = np.float32([[560,470],[730,470],[1100,720],[200,720]])      
apply_window_search = True
faulty_frame_count = 0
MAX_FAULTY_FRAMES = 3
NUM_PREV_FITS = 10
left_fit = []
right_fit = []
previous_left_fits = deque(maxlen=NUM_PREV_FITS)
previous_right_fits = deque(maxlen=NUM_PREV_FITS)
#previous good left curvatures
previous_left_curvs = deque(maxlen=NUM_PREV_FITS)
#previous good right curvatures
previous_right_curvs = deque(maxlen=NUM_PREV_FITS)

# Calibrate camera
mtx, dist = calibrate_camera()
# clear_test_dir()
#test_imgs()    
process_video()

[MoviePy] >>>> Building video project_output.mp4
[MoviePy] Writing video project_output.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [03:23<00:00,  6.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_output.mp4 

Wall time: 3min 24s
